In [1]:
import os
import pandas as pd
import re
from collections import ChainMap
import json

In [2]:
with open('../neuroscience/Papers_neuroscience.json', "r") as read_file:
    papers = json.load(read_file)

In [3]:
your_path = '../../../../../../Volumes/TOSHIBA EXT/Simplified_files'
print( os.listdir(your_path))
infile=your_path+'/paperReferences_simplified.txt'

['.DS_Store', 'affiliations.json', 'Authors_simplified.txt', 'journals.csv', 'paperAbstracts_simplified.txt', 'paperAuthors_simplified.txt', 'paperReferences_simplified.txt', 'Papers_simplified.txt']


In [4]:
listPapers=[int(p['id']) for p in papers]

In [5]:
thisDf=pd.DataFrame()
chunksize = 10 ** 6
for chunk in pd.read_csv(infile, chunksize=chunksize, names=['citing', 'cited'],sep=','):
    thisDf=thisDf.append(chunk[chunk['citing'].isin(listPapers)])
    thisDf=thisDf.append(chunk[chunk['cited'].isin(listPapers)])

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (0,1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
len(thisDf)

93594962

In [8]:
outfile='../neuroscience/citationNetwork.csv'
thisDf.to_csv(outfile)

In [9]:
allPapersCN=list(set(thisDf['citing'])|set(thisDf['cited']))

In [10]:
notInList=list(set(allPapersCN)-set(listPapers))

In [11]:
outFile='../neuroscience/temp/nextToSearch.txt'
ww=open(outFile,'w')
for el in notInList:
    ww.write(str(el)+'\n')
ww.close()

# ADD CITED and CITING papers

In [12]:
your_path = '../../../../../../Volumes/TOSHIBA EXT/Simplified_files'
print( os.listdir(your_path))
infile=your_path+'/Papers_simplified.txt'

['.DS_Store', 'affiliations.json', 'Authors_simplified.txt', 'journals.csv', 'paperAbstracts_simplified.txt', 'paperAuthors_simplified.txt', 'paperReferences_simplified.txt', 'Papers_simplified.txt']


In [13]:
newDf=pd.DataFrame()
chunksize = 5*10 ** 6
for chunk in pd.read_csv(infile, chunksize=chunksize, names=['id', 'feature','value'],sep=';'):
    newDf=newDf.append(chunk[chunk['id'].isin(notInList)])

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [14]:
redDf=newDf[newDf['feature']!=' http://www.w3.org/1999/02/22-rdf-syntax-ns#type']
redDf=newDf[newDf['feature']!=' rank']
redDf=newDf[newDf['feature']!=' estimatedCitationCount']

In [15]:
feat=list(set(redDf['feature']))

In [16]:
dictFeat={}
for f in feat:
    #dictFeat[f]={}
    df=redDf[redDf['feature']==f]
    dictFeat[f]=dict(zip(df['id'],df['value']))

In [17]:
ids=list(set(redDf['id']))

In [21]:
dictPapers=[]
for p in ids:
    thisdict={}
    thisdict['id']=p
    thisdict['queryLevel']=1
    for f in dictFeat.keys():
        try:
            thisdict[f]=dictFeat[f][p]
        except:
            continue
    dictPapers.append(thisdict)

In [22]:
dictPapers[0]

{'id': 2063597569,
 'queryLevel': 1,
 ' http://www.w3.org/1999/02/22-rdf-syntax-ns#type': ' http://purl.org/spar/fabio/JournalArticle',
 ' publisher': ' "Wiley"',
 ' volume': ' "18"',
 ' publicationDate': ' "2004-12-01"',
 ' title': ' "Simultaneous determination of D- and L-serine in rat brain microdialysis sample using a column-switching HPLC with fluorimetric detection."',
 ' endingPage': ' "819"',
 ' rank': ' "19997"',
 ' citationCount': ' "85"',
 ' referenceCount': ' "25"',
 ' appearsInJournal': ' 123979729',
 ' doi': '10.1002/BMC.394',
 ' created': ' "2016-06-24"',
 ' issueIdentifier': ' "10"',
 ' startingPage': ' "813"'}

In [23]:
papers[0]

{'id': 2038431747,
 ' created': ' "2016-06-24"',
 ' doi': '10.1162/NECO.1994.6.3.441',
 ' volume': ' "6"',
 ' appearsInJournal': ' 207023548',
 ' citationCount': ' "16"',
 ' issueIdentifier': ' "3"',
 ' title': ' "Topology learning solved by extended objects: A neural network model"',
 ' referenceCount': ' "7"',
 ' endingPage': ' "458"',
 ' startingPage': ' "441"',
 ' publicationDate': ' "1994-05-01"',
 ' publisher': ' "MIT Press"'}

In [24]:
for p in papers:
    p['queryLevel']=0

In [25]:
allpapers=papers+dictPapers

In [26]:
with open('../neuroscience/Papers_neuroscience_all.json', 'w') as fout:
    json.dump(allpapers , fout)